In [9]:
import numpy as np
import csv
from tqdm import tqdm
import networkx as nx

In [10]:
minp=200 #density threshold
N=5 #the step size of grids for each dimension.

# Load data

In [11]:
X=[]
filename = './Data_Q2/X.csv'
with open(filename) as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        X.append([int(it) for it in row])
X=np.array(X)

Construct the graph

In [12]:
M,d=X.shape

min_X=np.min(X,0)
max_X=np.max(X,0)


G=nx.Graph()

cell2point={}
point2cell={}

axis2cell={}
graph_num=0

for m in tqdm(range(M)):
    index=np.floor((X[m,:]-min_X)/(max_X-min_X)*N)
    axis=0
    for i in range(index.shape[0]):
        axis+=(N**i)*index[i]
    if axis in axis2cell.keys():
        cell2point[axis2cell[axis]].append(m)
        point2cell[m]=axis2cell[axis]
    else:
        axis2cell[axis]=graph_num
        G.add_node(graph_num)
        cell2point[graph_num]=[m]
        point2cell[m]=graph_num
        graph_num+=1

100%|███████████████████████████████| 286047/286047 [00:08<00:00, 34350.83it/s]


Add the edegs between these points that are more than the density threshold

In [13]:
keys=sorted(axis2cell.keys())
d=[N**i for i in range(10)]
for k1 in tqdm(range(keys.__len__())):
    if cell2point[axis2cell[keys[k1]]].__len__()>=minp:
        for k2 in range(k1+1,keys.__len__()):
            if keys[k2]==keys[k1]:
                continue
            tmp = abs(keys[k2] - keys[k1])
            if np.any(tmp==d) :
                G.add_edge(axis2cell[keys[k1]], axis2cell[keys[k2]])

100%|█████████████████████████████████████| 8587/8587 [00:14<00:00, 587.84it/s]


Find the max component in the graph

In [14]:
Gc = max(nx.connected_component_subgraphs(G), key=len)

Save the results

In [15]:
with open('Q2_output.csv','w', newline='') as f:
    writer=csv.writer(f)
    writer.writerow(['Result'])
    for m in tqdm(range(M)):
        if point2cell[m] in Gc.nodes:
            writer.writerow(['0'])
        else:
            writer.writerow(['1'])

100%|██████████████████████████████| 286047/286047 [00:00<00:00, 391533.82it/s]
